In [1]:
import os
import cv2
import numpy as np
import pytesseract
import urllib
from PIL import Image, ImageEnhance, ImageFilter

In [2]:
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

In [3]:
url = 'https://www.circleone.in/images/products_gallery_images/street-signs-03.jpg'

In [4]:
img = np.asarray(bytearray(urllib.request.urlopen(url).read()), dtype="uint8")
img = cv2.imdecode(img, cv2.IMREAD_COLOR)

In [5]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [6]:
ret, thresh1 = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

In [7]:
rect_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (105, 105))

In [8]:
dilation = cv2.dilate(thresh1, rect_kernel, iterations = 1)

In [9]:
contours, hierarchy = cv2.findContours(dilation, cv2.RETR_EXTERNAL,  
                                                 cv2.CHAIN_APPROX_NONE) 

In [10]:
text = []
for cnt in contours: 
    x, y, w, h = cv2.boundingRect(cnt) 
    rect = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cropped = img[y:y + h, x:x + w]
    file = open("recognized.txt", "a")
    text.append(pytesseract.image_to_string(cropped))

In [11]:
text.reverse()
text

['']

In [13]:
text = []
for cnt in contours: 
    x, y, w, h = cv2.boundingRect(cnt) 
    rect = cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cropped = img[y:y + h, x:x + w]
    file = open("recognized.txt", "a")
    text.append(pytesseract.image_to_string(cropped))

In [14]:
#Load Image
image = np.asarray(bytearray(urllib.request.urlopen(url).read()), dtype="uint8")
image = cv2.imdecode(image, cv2.IMREAD_COLOR)
im_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
im_blur = img_blur = cv2.GaussianBlur(im_gray,(5,5),0)
ret, im_thresh = cv2.threshold(im_blur, 100, 255, cv2.THRESH_OTSU | cv2.THRESH_BINARY_INV)

In [15]:
contours, _ = cv2.findContours(im_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 

In [16]:
text = []
for cnt in contours: 
    x, y, w, h = cv2.boundingRect(cnt) 
    rect = cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 255), 2)
    cropped = image[y:y + h, x:x + w]
    file = open("recognized.txt", "a")
    text.append(pytesseract.image_to_string(cropped))

In [17]:
# Convert to Hue-Saturation-Value format
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
lower = np.array([25, 225, 29])
upper = np.array([179, 255, 255])
mask = cv2.inRange(hsv, lower, upper)

In [18]:
#dilate to connect text characters, create horizontal kernel
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,3))
dilate = cv2.dilate(mask, kernel, iterations=5)

In [19]:
#Find contours and filter using aspect ratio, remove non-text contours by filling in the contour
cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    ar = w / float(h)
    if ar < 5:
        cv2.drawContours(dilate, [c], -1, (0,0,0), -1)

In [20]:
result = 255 - cv2.bitwise_and(dilate, mask)
data = pytesseract.image_to_string(result, lang='eng',config='--psm 6')

In [21]:
data

''

In [14]:
def nothing(x):
    pass

In [15]:
cv2.namedWindow('image')

In [16]:
cv2.createTrackbar('HMin', 'image', 0, 179, nothing)
cv2.createTrackbar('SMin', 'image', 0, 255, nothing)
cv2.createTrackbar('VMin', 'image', 0, 255, nothing)
cv2.createTrackbar('HMax', 'image', 0, 179, nothing)
cv2.createTrackbar('SMax', 'image', 0, 255, nothing)
cv2.createTrackbar('VMax', 'image', 0, 255, nothing)

In [17]:
cv2.setTrackbarPos('HMax', 'image', 179)
cv2.setTrackbarPos('SMax', 'image', 255)
cv2.setTrackbarPos('VMax', 'image', 255)

In [ ]:
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

In [ ]:
while(1):

    hMin = cv2.getTrackbarPos('HMin', 'image')
    sMin = cv2.getTrackbarPos('SMin', 'image')
    vMin = cv2.getTrackbarPos('VMin', 'image')
    hMax = cv2.getTrackbarPos('HMax', 'image')
    sMax = cv2.getTrackbarPos('SMax', 'image')
    vMax = cv2.getTrackbarPos('VMax', 'image')
    
    lower = np.array([hMin, sMin, vMin])
    upper = np.array([hMax, sMax, vMax])
    
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(image, image, mask=mask)
    
    if((phMin != hMin) | (psMin != sMin) | (pvMin != vMin) | (phMax != hMax) | (psMax != sMax) | (pvMax != vMax) ):
        print("(hMin = %d , sMin = %d, vMin = %d), (hMax = %d , sMax = %d, vMax = %d)" % (hMin , sMin , vMin, hMax, sMax , vMax))
        phMin = hMin
        psMin = sMin
        pvMin = vMin
        phMax = hMax
        psMax = sMax
        pvMax = vMax
    
    cv2.imshow('image', result)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
